In [1]:
import pandas as pd
import io
import re
import csv
import json
import numpy as np
from tqdm import tqdm
import math
import ast
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import seaborn as sns
import keras
from keras import backend as K
from progressbar import ProgressBar
import matplotlib.pyplot as plt
from keras import Sequential, Model
from random import randrange
from sklearn.preprocessing import StandardScaler
import sklearn.preprocessing as preprocessing
from keras.utils.vis_utils import plot_model
import time
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
import eli5
from eli5.sklearn import PermutationImportance

from plot_figures import save_model

pbar = ProgressBar()
tqdm.pandas()

Using TensorFlow backend.


In [2]:


max_blocks = 37 #(36 + 1 FC layer at the end)
#max_blocks = 36

nb_param =7
#nb_param =13
nb_hw_param = 6
#nb_param =12

inversed = True
only_inversed = False

BATCH_SIZE = 2048
SHUFFLE_BUFFER_SIZE = 100

DATASET_SIZE = 168308
train_size = int(0.90 * DATASET_SIZE) #135 805
test_size = int(0.10 * DATASET_SIZE) # 15 089

nb_training_batches = train_size //BATCH_SIZE +1
nb_test_batches = test_size //BATCH_SIZE +1

In [3]:

def from_np_array(array_string):
    array_string = ','.join(array_string.replace('[ ', '[').split())
    return np.asarray(ast.literal_eval(array_string))

# Load Proccessed Neural Networks (without HyperParameters)

In [4]:

path_raw_val = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_val_all.csv'
path_raw_train = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/parsed_nondups_train_all.csv'

#path_processed_val_nn = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/val_13.csv'
#path_processed_train_nn = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/train_13.csv'


path_processed_val_nn = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/val_7.csv'
path_processed_train_nn = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/train_7.csv'


In [5]:
#%pycache
tin= time.clock()

val = pd.read_csv(path_processed_val_nn,converters={'NN_dataframe': from_np_array})
train = pd.read_csv(path_processed_train_nn,converters={'NN_dataframe': from_np_array})

tfin= time.clock()
print(f'Total time : {(tfin-tin)/60} min')

Total time : 1.4375791333333332 min


# Complete to zeros (max shape)

In [6]:
def add_zero_blocks(arr):
    zero_blocks = np.zeros((max_blocks-arr.shape[0],arr.shape[1]))
    return np.append(arr, zero_blocks, axis=0)

val['NN_dataframe']= val['NN_dataframe'].apply(lambda x : add_zero_blocks(x))
train['NN_dataframe']= train['NN_dataframe'].apply(lambda x : add_zero_blocks(x))



# Get y_train, y_val


In [7]:
# To retrieve y_train / y_test

train_hw = pd.read_csv(path_raw_train)
val_hw = pd.read_csv(path_raw_val)

y_train = np.array(train_hw["total_power"].tolist())
y_train=y_train.reshape(y_train.shape[0],-1)

y_val = np.array(val_hw["total_power"].tolist())
y_val=y_val.reshape(y_val.shape[0],-1)

--------
# HW PARAM ALONE

In [8]:
X_train_hw_param  = train_hw[['mac_num', 'mac_array_num', 'data_bits', 'sram_size', 'max_filter_size']]
X_train_hw_param['tot_mac'] = X_train_hw_param['mac_num']*X_train_hw_param['mac_array_num']

X_val_hw_param  = val_hw[['mac_num', 'mac_array_num', 'data_bits', 'sram_size', 'max_filter_size']]
X_val_hw_param['tot_mac'] = X_val_hw_param['mac_num']*X_val_hw_param['mac_array_num']

X_train_hw_param.head()

,mac_num,mac_array_num,data_bits,sram_size,max_filter_size,tot_mac
0,118.0,2.0,256.0,179968.0,2048.0,236.0
1,87.0,2.0,512.0,99968.0,2048.0,174.0
2,124.0,2.0,512.0,99968.0,3072.0,248.0
3,80.0,2.0,1024.0,99968.0,1024.0,160.0
4,86.0,2.0,512.0,60000.0,1536.0,172.0


# Inv

In [9]:
def inv(x):
    return 1/(1+x)


if inversed : 
#nb_hw_param = 6
    nb_hw_param = 12
    X_train_hw_param['1/mac_num'] = X_train_hw_param['mac_num'].progress_apply(lambda x : inv(x))
    X_train_hw_param['1/mac_array_num'] = X_train_hw_param['mac_array_num'].progress_apply(lambda x : inv(x))
    X_train_hw_param['1/data_bits'] = X_train_hw_param['data_bits'].progress_apply(lambda x : inv(x))
    X_train_hw_param['1/sram_size'] = X_train_hw_param['sram_size'].progress_apply(lambda x : inv(x))
    X_train_hw_param['1/max_filter_size'] = X_train_hw_param['max_filter_size'].progress_apply(lambda x : inv(x))
    X_train_hw_param['1/tot_mac'] = X_train_hw_param['tot_mac'].progress_apply(lambda x : inv(x))


    X_val_hw_param['1/mac_num'] = X_val_hw_param['mac_num'].progress_apply(lambda x : inv(x))
    X_val_hw_param['1/mac_array_num'] = X_val_hw_param['mac_array_num'].progress_apply(lambda x : inv(x))
    X_val_hw_param['1/data_bits'] = X_val_hw_param['data_bits'].progress_apply(lambda x : inv(x))
    X_val_hw_param['1/sram_size'] = X_val_hw_param['sram_size'].progress_apply(lambda x : inv(x))
    X_val_hw_param['1/max_filter_size'] = X_val_hw_param['max_filter_size'].progress_apply(lambda x : inv(x))
    X_val_hw_param['1/tot_mac'] = X_val_hw_param['tot_mac'].progress_apply(lambda x : inv(x))

if only_inversed : 
    nb_hw_param = 6
    inv = ['1/mac_num','1/mac_array_num','1/data_bits','1/sram_size','1/max_filter_size','1/tot_mac']
    X_train_hw_param =X_train_hw_param.loc[:, inv]
    X_val_hw_param =X_val_hw_param.loc[:, inv]


100%|██████████| 16686/16686 [00:00<00:00, 703958.60it/s]


In [10]:
X_val_hw_param.head()

,mac_num,mac_array_num,data_bits,sram_size,max_filter_size,tot_mac,1/mac_num,1/mac_array_num,1/data_bits,1/sram_size,1/max_filter_size,1/tot_mac
0,37.0,2.0,128.0,199936.0,4098.0,74.0,0.026316,0.333333,0.007752,0.000005,0.000244,0.013333
1,91.0,2.0,256.0,119936.0,2048.0,182.0,0.010870,0.333333,0.003891,0.000008,0.000488,0.005464
2,74.0,2.0,256.0,139904.0,1024.0,148.0,0.013333,0.333333,0.003891,0.000007,0.000976,0.006711
3,70.0,2.0,512.0,80000.0,512.0,140.0,0.014085,0.333333,0.001949,0.000012,0.001949,0.007092
4,103.0,2.0,256.0,199936.0,4098.0,206.0,0.009615,0.333333,0.003891,0.000005,0.000244,0.004831


## HW Normalize

In [11]:
# normalize

#%pycache
# Process
scaler = preprocessing.StandardScaler().fit(X_train_hw_param)
X_train_hw_param_norm= scaler.transform(X_train_hw_param)
X_val_hw_param_norm = scaler.transform(X_val_hw_param )


## Datasets for HW param

In [12]:

X_hw_norm = np.concatenate((X_train_hw_param_norm, X_val_hw_param_norm))
X_hw = np.concatenate((X_train_hw_param, X_val_hw_param))
y = np.concatenate((y_train, y_val))

# Mix train/test
#NORMED
dataset_hw_norm = tf.data.Dataset.from_tensor_slices((X_hw_norm, y))
dataset_hw_norm= dataset_hw_norm.shuffle(SHUFFLE_BUFFER_SIZE)
#UNORMED
dataset_hw = tf.data.Dataset.from_tensor_slices((X_hw, y))
dataset_hw= dataset_hw.shuffle(SHUFFLE_BUFFER_SIZE)

# Split them
#NORMED
train_dataset_hw_norm =  dataset_hw_norm.take(train_size).batch(BATCH_SIZE)
test_dataset_hw_norm = dataset_hw_norm.skip(train_size).batch(BATCH_SIZE)

#UNORMODED
train_dataset_hw =  dataset_hw.take(train_size).batch(BATCH_SIZE)
test_dataset_hw = dataset_hw.skip(train_size).batch(BATCH_SIZE)


------------

# NN TREATEMENT ALONE

In [13]:


X_train_nn =np.array(train['NN_dataframe'].tolist())
X_val_nn = np.array(val['NN_dataframe'].tolist())

## NN Normalize

### Divde columns by standard deviation

In [14]:


means = []
std = []
X_train_nn_norm=X_train_nn
X_val_nn_norm = X_val_nn
for i in range(nb_param):
    #means.append(np.mean(X_train_nn[:,:,i]))
    #X_train_nn[:,:,i]=-means[i]
    std.append(np.std(X_train_nn[:,:,i]))
    X_train_nn[:,:,i]/= std[i]
    X_val_nn[:,:,i]/= std[i]  



### Bad normalisation

In [15]:
%pycache
# #Reshape
train_reshape= X_train_nn.reshape(X_train_nn.shape[0], -1)
val_reshape= X_val_nn.reshape(X_val_nn.shape[0], -1)

# # Process
scaler = preprocessing.StandardScaler().fit(train_reshape)
train_scaled= scaler.transform(train_reshape)
val_scaled = scaler.transform(val_reshape)


#Re-reshape
X_train_nn_norm= train_scaled.reshape(X_train_nn.shape)
X_val_nn_norm= val_scaled.reshape(X_val_nn.shape)

UsageError: Line magic function `%pycache` not found.


In [16]:
std

[290269479.2486268,
 502976.69976542174,
 134091.3240227188,
 135896.62164816633,
 313.5131629286966,
 12.21604009918021,
 0.39038205807298065]

In [17]:
pd.DataFrame(X_train_nn_norm[0]).head()

,0,1,2,3,4,5,6
0,0.021002,0.000501,1.122578,0.830749,0.009569,0.736736,0.0
1,0.007001,0.000179,0.841934,0.830749,0.028707,0.736736,0.0
2,0.006223,0.000159,0.841934,0.738444,0.028707,0.081860,0.0
3,0.192912,0.004499,0.748385,2.861469,0.025517,0.736736,0.0
4,0.868103,0.020544,2.899994,3.784524,0.395518,0.736736,0.0


## Datasets for NN

In [18]:

X_nn_norm = np.concatenate((X_train_nn_norm, X_val_nn_norm))
X_nn = np.concatenate((X_train_nn, X_val_nn))
y = np.concatenate((y_train, y_val))

# Mix test/train 
#NORMED
dataset_nn_norm = tf.data.Dataset.from_tensor_slices((X_nn_norm, y))
dataset_nn_norm= dataset_nn_norm.shuffle(SHUFFLE_BUFFER_SIZE)

#UNNORMED
dataset_nn = tf.data.Dataset.from_tensor_slices((X_nn, y))
dataset_nn.shuffle(SHUFFLE_BUFFER_SIZE)


# Split them
#NORMED
test_dataset_nn_norm = dataset_nn_norm.take(test_size).batch(BATCH_SIZE)
train_dataset_nn_norm = dataset_nn_norm.skip(test_size).batch(BATCH_SIZE)

#UNORMED
test_dataset_nn= dataset_nn.take(test_size).batch(BATCH_SIZE)
train_dataset_nn = dataset_nn.skip(test_size).batch(BATCH_SIZE)

# Full Dataset (HW_params + NN arrays)

In [19]:
#NORMED
train_dataset_norm = tf.data.Dataset.from_tensor_slices(((X_train_nn_norm, X_train_hw_param_norm), y_train))
test_dataset_norm = tf.data.Dataset.from_tensor_slices(((X_val_nn_norm, X_val_hw_param_norm), y_val))

#UNORMED
train_dataset = tf.data.Dataset.from_tensor_slices(((np.array(X_train_nn), np.array(X_train_hw_param)), y_train))
test_dataset = tf.data.Dataset.from_tensor_slices(((np.array(X_val_nn), np.array(X_val_hw_param)), y_val))


### Shuffle train/test dataset

In [20]:
#%pycache
# Mix between train/test (mix networks)
#NORMED
full_dataset_norm = train_dataset_norm.concatenate(test_dataset_norm)
full_dataset_norm = full_dataset_norm.shuffle(SHUFFLE_BUFFER_SIZE)

test_dataset_norm = full_dataset_norm.take(test_size)
train_dataset_norm = full_dataset_norm.skip(test_size)

train_dataset_norm = train_dataset_norm.batch(BATCH_SIZE)
test_dataset_norm = test_dataset_norm.batch(BATCH_SIZE)


#UNORMED
full_dataset = train_dataset.concatenate(test_dataset_norm)
full_dataset = full_dataset.shuffle(SHUFFLE_BUFFER_SIZE)

test_dataset= full_dataset.take(test_size)
train_dataset = full_dataset.skip(test_size)

train_dataset = train_dataset_norm.batch(BATCH_SIZE)
test_dataset = test_dataset_norm.batch(BATCH_SIZE)

### Small dataset

In [21]:
#%pycache
mini_batches = 2

train_dataset_small =train_dataset.take(mini_batches).batch(BATCH_SIZE)
test_dataset_small =test_dataset.take(mini_batches).batch(BATCH_SIZE)

train_dataset_small_norm =train_dataset_norm.take(mini_batches)
test_dataset_small_norm =test_dataset_norm.take(mini_batches)

# Get loss/model

## Models

In [34]:
name_test = 'model_1'

In [ ]:
def loss(target_power, predicted_power):
  return tf.math.abs((target_power - predicted_power)/target_power)

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-2,
    decay_steps=2,
    #decay_steps=nb_training_batches,
    decay_rate=0.95)


optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [56]:

 model_nn= tf.keras.Sequential([
       layers.GRU(16, return_sequences=True,input_shape=(max_blocks, nb_param)),
       #layers.LSTM(16, return_sequences=True),
       #layers.LSTM(32),
       layers.Conv1D(128, (3), activation='relu', padding="same"),
       #layers.Dense(32, activation='relu'),
       #layers.Dense(32, activation='relu'),
      #layers.Flatten()
 ])


model_hw = tf.keras.Sequential([
     layers.Dense(32, activation='relu', input_shape=(nb_hw_param,)),
     #layers.Dropout(0.1),
     #layers.Dense(32, activation='relu'),
     #layers.Dense(16, activation='relu'),
     #layers.Dense(16, activation='relu'),
     #layers.Dropout(0.1),
     layers.Dense(32, activation='linear')
 ])

#concat = tf.keras.layers.Concatenate()([model_nn.output, model_hw.output])
concat = tf.keras.layers.multiply([model_nn.output, model_nn.output])

output = tf.keras.layers.Dense(units=16, activation='relu')(concat)
#output = tf.keras.layers.Dense(units=16, activation='relu')(output)
#output = tf.keras.layers.Dense(units=16, activation='relu')(output)
output = tf.keras.layers.Dense(units=1, activation='relu')(output)
full_model = tf.keras.Model(inputs=[model_nn.input, model_hw.input], outputs=[output])

print(f'PARAMS : {full_model.count_params()}')

PARAMS : 9553


In [57]:
model_nn.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 37, 16)            1200      
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 37, 128)           6272      
Total params: 7,472
Trainable params: 7,472
Non-trainable params: 0
_________________________________________________________________


# Test entire model

In [61]:
full_epochs = 300

print(f'PARAMS : {full_model.count_params()}')
t4= time.clock()
full_model.compile(optimizer=optimizer, loss=loss)


history = full_model.fit(train_dataset_small_norm,
        validation_data=(test_dataset_small_norm),
        epochs=full_epochs,
       verbose=1,
 )


# history = full_model.fit(train_dataset_norm,
#          validation_data=(test_dataset_norm),
#         epochs=full_epochs,
#          verbose=1,
#  )

t5= time.clock()

print(f'Training Full Model for {full_epochs} epochs : {(t5-t4)/60} min')

PARAMS : 9553
Epoch 1/300
2/2 [==============================] - 9s 4s/step - loss: 0.9988 - val_loss: 0.0000e+00
Epoch 2/300
2/2 [==============================] - 2s 1s/step - loss: 0.9380 - val_loss: 0.9501
Epoch 3/300
2/2 [==============================] - 2s 1s/step - loss: 1.0905 - val_loss: 0.8720
Epoch 4/300
2/2 [==============================] - 2s 1s/step - loss: 0.8051 - val_loss: 0.8531
Epoch 5/300
2/2 [==============================] - 3s 1s/step - loss: 0.7576 - val_loss: 0.6886
Epoch 6/300
2/2 [==============================] - 2s 1s/step - loss: 0.6372 - val_loss: 0.7432
Epoch 7/300
2/2 [==============================] - 3s 1s/step - loss: 0.6744 - val_loss: 0.6522
Epoch 8/300
2/2 [==============================] - 2s 1s/step - loss: 0.5924 - val_loss: 0.6503
Epoch 9/300
2/2 [==============================] - 2s 928ms/step - loss: 0.5925 - val_loss: 0.6347
Epoch 10/300
2/2 [==============================] - 2s 910ms/step - loss: 0.5538 - val_loss: 0.6955
Epoch 11/300
2/

KeyboardInterrupt: 

## 10,785 params:
### 25 epochs : 56 min
### 15 epochs : 34 min

## 57,009 params
### 20 epochs : 101 min

## 143,345 params
### 20 epochs : 276 min

In [87]:
nb_final_epochs_for_mean = 2
print(f'PARAMS : {full_model.count_params()}')
name = f'{name_test}_{full_model.count_params()}_param'
save = True

nb_predictions = 1000
max_val_loss=53


path = "/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/results_best_models/RNN"
model = full_model
X_val_list = [X_val_nn_norm, X_val_hw_param_norm]
history = history

#model = full_model
#X_list = [X_val_hw_param_norm, X_val_nn_norm]

s#ave_model(path=path, model= model, history=history, X_list=X_val_list, y=y_val, name=name, nb_predictions  b_predictions, max_val_loss=max_val_loss, nb_final_epochs_for_mean = nb_final_epochs_for_mean, save=save)


t6= time.clock()


SyntaxError: EOL while scanning string literal (<ipython-input-87-d9b3ed997edd>, line 10)

In [34]:
name

'model_4_62273_param'

In [35]:
print(f'Total time : {(t6-tin)/60} min')

Total time : 1.4952906166666666 min


In [36]:
len(X_val_list)

2

In [37]:

#X=list(zip(X_val_nn_norm, X_val_hw_param_norm))
X= X_val_list
y = y_val


def score(X_list,y):
    losses=[]
    inputs =[]
    for i in range(len(X_list[0])):
        for x in range(len(X_list)): # whether one or multiple inputs
            input_x=np.array(X_list[x])[i]
            input_x=input_x.reshape((1,*input_x.shape))
            inputs.append(tf.convert_to_tensor(input_x))
        pred = model.predict(inputs)
        losses.append(loss(y[i],pred))
    return losses

perm = PermutationImportance(full_model, random_state=1, scoring=score).fit(X,y)
eli5.show_weights(perm, feature_names = X.columns.tolist())

ValueError: could not broadcast input array from shape (16686,37,7) into shape (16686)

In [38]:
def score(X_list,y):
    losses=[]
    inputs =[]
    for i in range(len(X_list[0])):
        for x in range(len(X_list)): # whether one or multiple inputs
            input_x=np.array(X_list[x])[i]
            input_x=input_x.reshape((1,*input_x.shape))
            inputs.append(tf.convert_to_tensor(input_x))
        pred = model.predict(inputs)
        losses.append(loss(y[i],pred))
    return losses

score(X_val_list[:10],y_val[:10])

IndexError: index 10 is out of bounds for axis 0 with size 10

In [39]:
base_score, score_decreases = get_score_importances(score, X_val_list, y_val)

NameError: name 'get_score_importances' is not defined

In [40]:
(X_val_nn_norm, X_val_hw_param_norm).shape

AttributeError: 'tuple' object has no attribute 'shape'

In [41]:
np.asarray((X_val_nn_norm, X_val_hw_param_norm))

ValueError: could not broadcast input array from shape (16686,37,7) into shape (16686)

In [42]:
import numpy as np